#### (1) Reasonable review growth (8%)
#### (2) Reasonable PAT growth
#### (3) Reasonable NPAT growth
#### (4) Avg receivables < 20% of revenue (or) Avg receivables < Revenue growth
#### (5) Latest ROE > Avg ROE
#### (6) Average profit capital employed > 15
#### (7) CFO growth higher than Revenue growth
#### (8) CFO growth > 10%
#### (9) CCFO_to_CNPAT > 0.75
#### (10) Debt < 20% of Revenue (or) Debt growth < Revenue growth
#### (11) Latest EBIT to NetProf higher than Avg.EBIT to NetProf
#### (12) OPM >0 and Book per Share >0 and Prom.Hold >45 and ROE > 10 and EPS> 0

In [7]:
import pandas as pd
import yahoo_fin.stock_info as si
import math

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 100)

nse_500=pd.read_csv(r"C:\Users\Deepan\Documents\GitHub\deenisha\ind_nifty500list.csv")

##################################################
#			Declaration
###################################################

rev_score=0
pat_score=0
nprof_score=0
avg_rec_score=0
roe_score=0
avg_prof_cap_emp_score=0
cfo_score=0
cfo_gt_10_score=0
ccflo_to_cnpat_score=0
debt_score=0
base_validation=0
selected_list=[]
val_ebit_to_net_prof=0

def my_stock_price(stock_nm):

 
    income_statement = si.get_income_statement(stock_nm)
    balance_sheet = si.get_balance_sheet(stock_nm)
    cash_flow_statement = si.get_cash_flow(stock_nm)
    
    income_statement.fillna(0, inplace=True)
    income_statement=income_statement.div(10000000).astype(int)
    
    balance_sheet.fillna(0, inplace=True)
    balance_sheet=balance_sheet.div(10000000).astype(int)
    
    cash_flow_statement.fillna(0, inplace=True)
    cash_flow_statement=cash_flow_statement.div(10000000).astype(int)
    
    income_statement_df=income_statement.transpose()
    balance_sheet_df=balance_sheet.transpose()
    cash_flow_statement_df=cash_flow_statement.transpose()
    
    revenue=income_statement_df['totalRevenue']
    pat=income_statement_df['incomeBeforeTax']
    nprof=income_statement_df['netIncomeFromContinuingOps']
    
    if "netReceivables" in balance_sheet_df:
        avg_recei=balance_sheet_df['netReceivables'].mean()
        avg_recei_latest=balance_sheet_df['netReceivables']
    else:
        avg_recei_df = revenue
        avg_recei_df = avg_recei_df.replace(avg_recei_df, 0)
        avg_recei=avg_recei_df.mean()
        avg_recei_latest=avg_recei_df
    
    #avg_recei=balance_sheet_df['netReceivables'].mean()
    avg_rev_20_pct=(income_statement_df['totalRevenue'].mean()*(20/100))
    #avg_recei_latest=balance_sheet_df['netReceivables']
    
    tot_share_hold=balance_sheet_df['totalStockholderEquity'].sort_index()
    net_income=income_statement_df['netIncome'].sort_index()
    
    roe=(net_income/tot_share_hold)*100
    roe_mean=roe.mean()
    
    
    np_share_hold=income_statement_df['netIncomeApplicableToCommonShares'].sort_index()
    prof_cap_emp=(np_share_hold/tot_share_hold)*100
    avg_prof_cap_emp=prof_cap_emp.mean()
    
    cfo=cash_flow_statement_df['totalCashFromOperatingActivities']
    CCFO=cfo.sum()
    CNPAT=income_statement_df['netIncomeApplicableToCommonShares'].sum()
    CCFO_to_CNPAT=round(CCFO/CNPAT,2)
    
    if "shortLongTermDebt" in balance_sheet_df:
        short_debt=balance_sheet_df['shortLongTermDebt']
    else:
        #print ("notfound")
        short_debt = revenue
        short_debt = short_debt.replace(short_debt, 0)
    
        
    if "longTermDebt" in balance_sheet_df:
        long_debt=balance_sheet_df['longTermDebt']
    else:
        long_debt = revenue
        long_debt = long_debt.replace(long_debt, 0)
    
    total_debt=short_debt+long_debt
	
	
    ebit=income_statement_df['ebit'].sort_index()
    net_prof_sort=income_statement_df['netIncomeApplicableToCommonShares'].sort_index()
    
    ebit_to_net_prof=(ebit/net_prof_sort)
    avg_ebit_to_net_prof=ebit_to_net_prof.mean()
	
	
    # sorting.......................
    
    revenue.sort_index()
    pat.sort_index()
    nprof.sort_index()
    avg_recei_latest.sort_index()
    cfo.sort_index()
    total_debt.sort_index()
    
    
    """ Values updates"""
    
    end_value = float(revenue.iloc[0])
    start_value = float(revenue.iloc[-1])
    num_periods = len(revenue)
    
    if end_value < 0:
        end_value=0.001
    else:
        end_value=end_value
        
    if start_value <= 0:
        start_value=0.001
    else:
        start_value=start_value
    
    if start_value == end_value:
        rev_grw=0
    else:
        rev_grw=((end_value / start_value) ** (1 / (num_periods )) - 1)*100
    
 
    end_value = float(pat.iloc[0])
    start_value = float(pat.iloc[-1])
    num_periods = len(pat)
    
    if end_value < 0:
        end_value=0.001
    else:
        end_value=end_value
        
    if start_value <= 0:
        start_value=0.001
    else:
        start_value=start_value
        
    if start_value == end_value:
        pat_grw=0
    else:
        pat_grw=((end_value / start_value) ** (1 / (num_periods )) - 1)*100
    
    
    end_value = float(nprof.iloc[0])
    start_value = float(nprof.iloc[-1])
    num_periods = len(nprof)
    
    if end_value < 0:
        end_value=0.001
    else:
        end_value=end_value
        
    if start_value <= 0:
        start_value=0.001
    else:
        start_value=start_value
    if start_value == end_value:
        nprof_grw=0
    else:
        nprof_grw=((end_value / start_value) ** (1 / (num_periods )) - 1)*100
    
    
    
    end_value = float(nprof.iloc[0])
    start_value = float(nprof.iloc[-1])
    num_periods = len(nprof)
    
    if end_value < 0:
        end_value=0.001
    else:
        end_value=end_value
        
    if start_value <= 0:
        start_value=0.001
    else:
        start_value=start_value
    
    if start_value == end_value:
        nprof_grw=0
    else:
        nprof_grw=((end_value / start_value) ** (1 / (num_periods )) - 1)*100
    
    
    end_value = float(avg_recei_latest.iloc[0])
    start_value = float(avg_recei_latest.iloc[-1])
    num_periods = len(avg_recei_latest)
    
    if end_value < 0:
        end_value=0.001
    else:
        end_value=end_value
        
    if start_value <= 0:
        start_value=0.001
    else:
        start_value=start_value
    
    if start_value == end_value:
        avg_recei_grw=0
    else:
        avg_recei_grw=((end_value / start_value) ** (1 / (num_periods )) - 1)*100
    
    latest_roe = float(roe.iloc[-1])
    
    end_value = float(cfo.iloc[0])
    start_value = float(cfo.iloc[-1])
    num_periods = len(cfo)
    
    if end_value < 0:
        end_value=0.001
    else:
        end_value=end_value
        
    if start_value <= 0:
        start_value=0.001
    else:
        start_value=start_value
    if start_value == end_value:
        cfo_grw=0
    else:
        cfo_grw=((end_value / start_value) ** (1 / (num_periods )) - 1)*100
    
    
    
    
    
    end_value = float(total_debt.iloc[0])
    start_value = float(total_debt.iloc[-1])
    num_periods = len(total_debt)
    
    if start_value == 0:
        start_value=0.01
    else:
        start_value=start_value
        
    if start_value == end_value:
        total_debt_grw=0
    else:
        total_debt_grw=((end_value / start_value) ** (1 / (num_periods )) - 1)*100
    
    
    
    total_debt_mean=total_debt.mean()
    total_rev_mean=revenue.mean()
    total_rev_mean_20_pct=total_rev_mean*(20/100)
    
    """ Values updates"""
    
    rev_simple=rev_grw-(rev_grw*(5/100))
    
    if rev_simple > 8:
        rev_score=1
    else:
        rev_score=0
        
    if pat_grw > (rev_simple - (rev_simple*(10/100))):
        pat_score=1
    else:
        pat_score=0
        
    if nprof_grw > (rev_simple - (rev_simple*(10/100))):
        nprof_score=1
    else:
        nprof_score=0
        
    if avg_recei < avg_rev_20_pct or avg_recei_grw <=(rev_simple - (rev_simple*(9/100))):
        avg_rec_score=1
    else:
        avg_rec_score=0
        
    if latest_roe > (roe_mean - (roe_mean*(10/100))):
        roe_score=1
    else:
        roe_score=0
        
    if avg_prof_cap_emp > 15 :
        avg_prof_cap_emp_score=1
    else:
        avg_prof_cap_emp_score=0
        
    if cfo_grw > (rev_simple - (rev_simple*(10/100))):
        cfo_score=1
    else:
        cfo_score=0
        
    if cfo_grw > 10:
        cfo_gt_10_score=1
    else:
        cfo_gt_10_score=0
        
    if CCFO_to_CNPAT > 0.75:
        ccflo_to_cnpat_score=1
    else:
        ccflo_to_cnpat_score=0
        
    if total_debt_mean < total_rev_mean_20_pct or total_debt_grw <=(rev_simple - (rev_simple*(10/100))) or total_debt_grw <0 :
        debt_score=1
    else:
        debt_score=0
		
    latest_ebit_to_net_prof = float(ebit_to_net_prof.iloc[-1])
    if latest_ebit_to_net_prof >= (avg_ebit_to_net_prof - (avg_ebit_to_net_prof*15/100)):
        val_ebit_to_net_prof=1
    else:
        val_ebit_to_net_prof=0
    #stock_nm="NESTLEIND.NS"
    stock_stats = si.get_stats(stock_nm)
    opm=float(stock_stats[stock_stats.Attribute=='Profit Margin'].Value.item().replace('%', ''))
    book_per_share=float(stock_stats[stock_stats.Attribute=='Book Value Per Share (mrq)'].Value.item())
    
    if isinstance(stock_stats[stock_stats.Attribute=='% Held by Insiders 1'].Value.item(), str):
        promoter_hold=float(stock_stats[stock_stats.Attribute=='% Held by Insiders 1'].Value.item().replace('%', ''))
    elif math.isnan(stock_stats[stock_stats.Attribute=='% Held by Insiders 1'].Value.item()):
        promoter_hold=46
    else:
        promoter_hold=float(stock_stats[stock_stats.Attribute=='% Held by Insiders 1'].Value.item().replace('%', ''))
        
    #promoter_hold=float(stock_stats[stock_stats.Attribute=='% Held by Insiders 1'].Value.item().replace('%', ''))
    
    if isinstance(stock_stats[stock_stats.Attribute=='Return on Equity (ttm)'].Value.item(), str):
        roe=float(stock_stats[stock_stats.Attribute=='Return on Equity (ttm)'].Value.item().replace('%', ''))
    elif math.isnan(stock_stats[stock_stats.Attribute=='Return on Equity (ttm)'].Value.item()):
        roe=10
    else:
        roe=float(stock_stats[stock_stats.Attribute=='Return on Equity (ttm)'].Value.item().replace('%', ''))
    eps=float(stock_stats[stock_stats.Attribute=='Diluted EPS (ttm)'].Value.item())   
    
    if opm >=0 and book_per_share>=0 and promoter_hold>=45 and roe>=10 and eps>=0 :
        #print(stock_nm,opm,book_per_share,promoter_hold,roe,eps)
        base_validation=1
    else:
        base_validation=0

    total_score=(rev_score+pat_score+nprof_score+avg_rec_score+roe_score+avg_prof_cap_emp_score+cfo_score+cfo_gt_10_score+ccflo_to_cnpat_score+debt_score+val_ebit_to_net_prof)
    
    if total_score >=9 and base_validation>0 :
        print (stock_nm, "Good For Investment" , total_score)
        selected_list.append(stock_nm)
    else:
        print (stock_nm,"failed")


In [11]:
#Nifty 50
#stock_list=["ONGC.NS","GAIL.NS","MARUTI.NS","KOTAKBANK.NS","HEROMOTOCO.NS","NESTLEIND.NS","ULTRACEMCO.NS","HINDUNILVR.NS","TITAN.NS","ITC.NS","IOC.NS","LT.NS","RELIANCE.NS","BPCL.NS","CIPLA.NS","COALINDIA.NS","BRITANNIA.NS","EICHERMOT.NS","AXISBANK.NS","ADANIPORTS.NS","HDFCLIFE.NS","BAJFINANCE.NS","BAJAJ-AUTO.NS","M&M.NS","TECHM.NS","INFY.NS","TCS.NS","BHARTIARTL.NS","GRASIM.NS","HDFCBANK.NS","SUNPHARMA.NS","ASIANPAINT.NS","NTPC.NS","HDFC.NS","SBILIFE.NS","DIVISLAB.NS","JSWSTEEL.NS","ICICIBANK.NS","HCLTECH.NS","BAJAJFINSV.NS","POWERGRID.NS","DRREDDY.NS","SBIN.NS","SHREECEM.NS","TATASTEEL.NS","HINDALCO.NS","UPL.NS","TATAMOTORS.NS","WIPRO.NS","INDUSINDBK.NS"]

#Nifty Next 50
#stock_list=["DMART.NS","ICICIGI.NS","COLPAL.NS","ABBOTINDIA.NS","IGL.NS","PGHH.NS","BERGEPAINT.NS","HDFCAMC.NS","DABUR.NS","PIDILITIND.NS","GODREJCP.NS","NAUKRI.NS","BAJAJHLDNG.NS","TORNTPHARM.NS","BIOCON.NS","MARICO.NS","HINDPETRO.NS","PEL.NS","TATACONSUM.NS","PETRONET.NS","ADANIGREEN.NS","LUPIN.NS","AUROPHARMA.NS","CADILAHC.NS","ACC.NS","ALKEM.NS","MUTHOOTFIN.NS","OFSS.NS","GICRE.NS","SIEMENS.NS","BOSCHLTD.NS","MCDOWELL-N.NS","AMBUJACEM.NS","LTI.NS","SBICARD.NS","UBL.NS","DLF.NS","ADANITRANS.NS","ICICIPRULI.NS","NMDC.NS","BANDHANBNK.NS","MOTHERSUMI.NS","PFC.NS","CONCOR.NS","INDIGO.NS","PNB.NS","INDUSTOWER.NS","HINDZINC.NS","HAVELLS.NS","BANKBARODA.NS"]

#Mid-cap-100
#stock_list=["EMAMILTD.NS","VGUARD.NS","CROMPTON.NS","ATGL.NS","ENDURANCE.NS","MGL.NS","ISEC.NS","PFIZER.NS","AJANTPHARM.NS","BATAINDIA.NS","AUBANK.NS","TVSMOTOR.NS","MFSL.NS","POLYCAB.NS","GODREJIND.NS","OIL.NS","CASTROLIND.NS","EDELWEISS.NS","ABFRL.NS","CESC.NS","FRETAIL.NS","NATCOPHARM.NS","OBEROIRLTY.NS","PRESTIGE.NS","ADANIENT.NS","COROMANDEL.NS","IRCTC.NS","VBL.NS","FORTIS.NS","SANOFI.NS","IPCALAB.NS","GLENMARK.NS","RECLTD.NS","MINDTREE.NS","SUNTV.NS","MRF.NS","CHOLAFIN.NS","APLLTD.NS","SYNGENE.NS","YESBANK.NS","PIIND.NS","RAMCOCEM.NS","WHIRLPOOL.NS","DALBHARAT.NS","ESCORTS.NS","GODREJAGRO.NS","JUBLFOOD.NS","TATAPOWER.NS","COFORGE.NS","SRF.NS","LALPATHLAB.NS","IDFCFIRSTB.NS","GODREJPROP.NS","GUJGASLTD.NS","L&TFH.NS","BALKRISIND.NS","INDHOTEL.NS","EXIDEIND.NS","UNIONBANK.NS","DHANI.NS","AARTIIND.NS","NAM-INDIA.NS","LTTS.NS","HUDCO.NS","RAJESHEXPO.NS","NAVINFLUOR.NS","AMARAJABAT.NS","TATACHEM.NS","RBLBANK.NS","GSPL.NS","TORNTPOWER.NS","MPHASIS.NS","BHEL.NS","PAGEIND.NS","IDEA.NS","BHARATFORG.NS","BBTC.NS","SRTRANSFIN.NS","ASHOKLEY.NS","JINDALSTEL.NS","FEDERALBNK.NS","BANKINDIA.NS","VOLTAS.NS","JSWENERGY.NS","BEL.NS","ZEEL.NS","M&MFIN.NS","CUB.NS","LICHSGFIN.NS","MANAPPURAM.NS","ABCAPITAL.NS","IBULHSGFIN.NS","CANBK.NS","SAIL.NS","GMRINFRA.NS","TRENT.NS","CUMMINSIND.NS","APOLLOHOSP.NS","NATIONALUM.NS","APOLLOTYRE.NS"]

#small-cap-100
#stock_list=["CSBBANK.NS","TV18BRDCST.NS","IBREALEST.NS","RCF.NS","DELTACORP.NS","IOLCP.NS","BLUESTARCO.NS","BASF.NS","IDFC.NS","KAJARIACER.NS","LEMONTREE.NS","TATAELXSI.NS","CENTURYPLY.NS","EQUITAS.NS","CANFINHOME.NS","JBCHEPHARM.NS","PGHL.NS","VAKRANGEE.NS","ASTERDM.NS","STAR.NS","VIPIND.NS","UJJIVANSFB.NS","AMBER.NS","IRB.NS","GODFRYPHLP.NS","DCBBANK.NS","KALPATPOWR.NS","INDIANB.NS","EIDPARRY.NS","PNBHOUSING.NS","PERSISTENT.NS","KEI.NS","TRIDENT.NS","AVANTIFEED.NS","JKLAKSHMI.NS","PVR.NS","INOXLEISUR.NS","CENTURYTEX.NS","LUXIND.NS","IRCON.NS","BEML.NS","CHAMBLFERT.NS","HEIDELBERG.NS","IEX.NS","INDIAMART.NS","THYROCARE.NS","LINDEINDIA.NS","SOBHA.NS","DEEPAKNTR.NS","SUVENPHAR.NS","NBCC.NS","RALLIS.NS","BIRLACORPN.NS","CEATLTD.NS","FDC.NS","AFFLE.NS","MCX.NS","WOCKPHARMA.NS","KARURVYSYA.NS","FSL.NS","DIXON.NS","UJJIVAN.NS","OMAXE.NS","BAJAJELEC.NS","SUNTECK.NS","GMMPFAUDLR.NS","SPARC.NS","TIMKEN.NS","STLTECH.NS","RVNL.NS","ENGINERSIN.NS","WELSPUNIND.NS","ALOKINDS.NS","RITES.NS","COCHINSHIP.NS","SONATSOFTW.NS","SPICEJET.NS","MIDHANI.NS","KEC.NS","HEG.NS","KSCL.NS","IDBI.NS","CDSL.NS","WELCORP.NS","MOIL.NS","GRAPHITE.NS","BALRAMCHIN.NS","GNFC.NS","WESTLIFE.NS","GRANULES.NS","BDL.NS","RADICO.NS","INDIACEM.NS","DCMSHRIRAM.NS","CYIENT.NS","JUSTDIAL.NS","DBL.NS","LAURUSLABS.NS","RAIN.NS","AEGISCHEM.NS"]

for stock in range(0,len(stock_list)):
    my_stock_price(stock_list[stock])
    

PRESTIGE.NS failed
ADANIENT.NS failed
COROMANDEL.NS Good For Investment 10
IRCTC.NS failed
VBL.NS failed
FORTIS.NS failed
SANOFI.NS Good For Investment 9
IPCALAB.NS Good For Investment 9
GLENMARK.NS failed
RECLTD.NS failed
MINDTREE.NS failed
SUNTV.NS failed
MRF.NS failed
CHOLAFIN.NS failed
APLLTD.NS Good For Investment 9
SYNGENE.NS failed
YESBANK.NS failed
PIIND.NS failed
RAMCOCEM.NS failed
WHIRLPOOL.NS failed
DALBHARAT.NS failed
ESCORTS.NS Good For Investment 9
GODREJAGRO.NS failed
JUBLFOOD.NS failed
TATAPOWER.NS failed
COFORGE.NS failed
SRF.NS failed
LALPATHLAB.NS Good For Investment 10
IDFCFIRSTB.NS failed
GODREJPROP.NS failed
GUJGASLTD.NS Good For Investment 10
L&TFH.NS failed
BALKRISIND.NS failed
INDHOTEL.NS failed
EXIDEIND.NS failed
UNIONBANK.NS failed
DHANI.NS failed
AARTIIND.NS Good For Investment 9
NAM-INDIA.NS failed
LTTS.NS Good For Investment 10
HUDCO.NS failed
RAJESHEXPO.NS failed
NAVINFLUOR.NS failed
AMARAJABAT.NS Good For Investment 9
TATACHEM.NS failed
RBLBANK.NS failed